1 - Create a repo in Github and clone it on local drive
2 - Set up Python environnement
3 - Create core folder to keep my functions
4 - Create data folder
5 - function summarise
6 - check for quick linear regression
 python3 -m ipykernel install --user --name venv


# Packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# Personal Packages

In [ ]:
from core.helpers import summarize
from core.helpers import transform_df_for_model
from core.helpers import sklearn_sm

# Linear regression : 1 variable
## Simple way

In [ ]:
Boston = pd.read_csv("data/Boston.csv")
df = pd.DataFrame({'intercept': np.ones(Boston.shape[0]),
                  'lstat': Boston['lstat']})
y = Boston['medv']
model = sm.OLS(y, df)
results = model.fit()
summarize(results)

## Using my own function

In [ ]:
X = transform_df_for_model(df,['lstat'])
y = Boston['medv']
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

In [ ]:
results.summary()

### Predictions

In [ ]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]})
variables = ['lstat']
newX = transform_df_for_model(new_df, variables)
newX

In [ ]:
new_predictions = results.get_prediction(newX)
print(new_predictions.predicted_mean)
new_predictions.conf_int(alpha=0.05)

Prediction interval

In [ ]:
new_predictions.conf_int(obs=True, alpha=0.05)

# Linear regression : 2 variables

In [ ]:
Boston = pd.read_csv("data/Boston.csv")
X = transform_df_for_model(Boston,['lstat', 'age'])
model1 = sm.OLS(y, X)
results1 = model1.fit()
summarize(results1)

In [ ]:
terms = Boston.columns.drop("medv")
X = transform_df_for_model(Boston, terms)
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

## with interactions

In [ ]:
X = transform_df_for_model(Boston, ['lstat','age'],interactions = [('lstat', 'age')])
model2 = sm.OLS(y, X)

results2 = model2.fit()
summarize(model2.fit())
results2.summary()

How to use anova_lm
anova(reduced,full)
Ho no need the full model


In [ ]:
anova_lm(results1,results2)

In [ ]:
Carseats = pd.read_csv("data/Carseats.csv")

allvars = list(Carseats.columns.drop('Sales'))
y = Carseats['Sales']

X = transform_df_for_model(Carseats, allvars,[('Income', 'Advertising'),
                   ('Price', 'Age')])
model = sm.OLS(y, X)
summarize(model.fit())

# Split


In [ ]:
Auto = pd.read_csv('data/Auto.csv')
Auto_train, Auto_test = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)

## use of partial

In [ ]:
hp_mm = partial(transform_df_for_model, terms = ['horsepower'])
X_train = hp_mm(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [ ]:
X_test = hp_mm(Auto_test)
y_test = Auto_test['mpg']
test_pred = results.predict(X_test)
np.mean((y_test - test_pred)**2).item()

## Function to compute MSE

In [ ]:
def evalMSE(terms,
            response,
            train,
            test):

   mm_transform = partial(transform_df_for_model, terms = terms)
   X_train = mm_transform(train)
   y_train = train[response]

   X_test = mm_transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2).item()

In [ ]:
evalMSE(terms =['horsepower','cylinders'],response ='mpg',train = Auto_train, test=Auto_test)

In [ ]:
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']

hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

In [ ]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class StatsmodelsLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model_ = None

    def fit(self, X, y):
        X = sm.add_constant(X)  # Ajout d'une constante pour l'intercept
        self.model_ = sm.OLS(y, X).fit()
        return self

    def predict(self, X):
        X = sm.add_constant(X)  # Ajout de l'intercept pour la prédiction
        return self.model_.predict(X)

    def summary(self):
        return self.model_.summary()

In [ ]:
# 📌 Données factices
data = pd.read_csv("data/Boston.csv")

# 📌 Variables et interactions
terms = ['lstat', 'age']

y = data['medv']


# 📌 Construction du pipeline
pipeline = make_pipeline(
    CustomDataTransformer(terms),
    StatsmodelsLinearRegression()
)

# 📌 Entraînement du modèle
pipeline.fit(data, y)

# 📌 Prédiction
predictions = pipeline.predict(data)

# 📌 Affichage du résumé de la régression
print(pipeline.named_steps["statsmodelslinearregression"].summary())